In [0]:
%python
#create table using the python
spark.sql("create database demo")
spark.sql("use demo")
df = spark.read.csv("/FileStore/tables/Order.csv", header=True, inferSchema=True, sep =',')
#df.write.mode("overwrite").partitionBy('Region').format("parquet").saveAsTable("demo.order")

In [0]:
df.creaeteOrRemplaceRempVie("Order")
df1 = spark.sql("select * from order grouby ")
df1.write.save("")

In [0]:
df1 = df.select("sfsd")
df2 =df1.grouBy("")
df2.write.save("")

In [0]:
#Create Delta Table

In [0]:
# This willsave the df in delta format but wont directly create a table in Databse
df.write.format("delta").save("/order_delta")

# This willsave the df in delta format and create a table in Databse (Internal Table)

df.write.format("delta").mode("overwrite").saveAsTable("/order_tab")

In [0]:
%sql

-- Create external table using the SQL way

CREATE TABLE demo.order_ext
USING DELTA
LOCATION '/order_tab'

In [0]:
%sql
SELECT * FROM f1_demo.order_ext

In [0]:
%sql
DESCRIBE HISTORY demo.order_de

Create managed table

In [0]:
%sql
CREATE OR REPLACE TABLE default.Order (
  id INT,
  firstName STRING,
  middleName STRING,
  lastName STRING,
  gender STRING,
  birthDate TIMESTAMP,
  ssn STRING,
  salary INT
) USING DELTA

create  table with Path

In [0]:
%sql
CREATE OR REPLACE TABLE delta.`/tmp/delta/people10m` (
  id INT,
  firstName STRING,
  middleName STRING,
  lastName STRING,
  gender STRING,
  birthDate TIMESTAMP,
  ssn STRING,
  salary INT
) USING DELTA

Create table using the partition

In [0]:
%sql

CREATE TABLE default.people10m (
  id INT,
  firstName STRING,
  middleName STRING,
  lastName STRING,
  gender STRING,
  birthDate TIMESTAMP,
  ssn STRING,
  salary INT
)
USING DELTA
PARTITIONED BY (gender)

Fix the number of records to be written on each file

In [0]:
df.write.format("delta") \
  .mode("append") \
  .option("maxRecordsPerFile", "10000") \
  .save("/tmp/delta/people10m")

Convert normal table to delta table

In [0]:
CREATE TABLE events USING PARQUET OPTIONS (path '/data-pipeline/')
CONVERT TO DELTA events

In [0]:
spark.read \
  .format("delta") \
  .load("/the/delta/table/path") \
  .withColumnRenamed("column name", "column-name") \
  .write \
  .format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .save("/the/delta/table/path")

Add / Rename column

In [0]:
%sql
ALTER TABLE table_name RENAME COLUMN old_col_name TO new_col_name
ALTER TABLE table_name DROP COLUMN col_name
ALTER TABLE table_name ADD COLUMNS (col_name data_type [COMMENT col_comment] [FIRST|AFTER colA_name], ...)

Insert Data

In [0]:
%sql
INSERT INTO default.people10m SELECT * FROM morePeople

Time Travel

In [0]:
%sql
-- Check the history
DESCRIBE HISTORY '/data/events/'  
DESCRIBE HISTORY eventsTable

--Restore table to specific version
RESTORE TABLE db.target_table TO VERSION AS OF <version>
RESTORE TABLE delta.`/data/target/` TO TIMESTAMP AS OF <timestamp>

In [0]:
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")



The "optimize" command in Databricks is used to optimize the layout of data in a table or partition, which helps to improve query performance. Essentially, this command reorganizes the data in the table or partition in a way that makes it easier and faster for the database to access and retrieve the information you need.

Think of it like reorganizing your closet. By putting similar items together, it becomes easier and faster to find what you need. The same is true for optimizing data in a database, but on a much larger scale. By using the "optimize" command, Databricks can make sure that the data is organized in a way that allows for efficient retrieval during query operations.

In [0]:
OPTIMIZE 

The data files backing a Delta table are never deleted automatically; data files are deleted only when you run VACUUM. VACUUM does not delete Delta log files; log files are automatically cleaned up after checkpoints are written.

In [0]:
VACUUM eventsTable   -- vacuum files not required by versions older than the default retention period

VACUUM '/data/events' -- vacuum files in path-based table


Z-Ordering

Z-ordering, or Z-order indexing, refers to the way in which records are stored in a data structure to optimize for query performance. The concept is that data is stored in such a way that allows for efficient search and retrieval of records based on their position in a space-filling curve. In Databricks, Z-ordering is used to sort data in a distributed environment for improved query performance.

In [0]:
OPTIMIZE table_name ZORDER BY (column_name)